In [1]:
import numpy as np
import pandas as pd
from scipy import sparse

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.metrics import log_loss, roc_auc_score

import matplotlib.pyplot as plt
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
PATH = '/content/drive/MyDrive/HSE/RecSys'

In [4]:
%%time
df = pd.read_csv(os.path.join(PATH,'data.csv'))
df.drop(['oaid_hash',  'rate0', 'rate1'], axis=1, inplace=True)
df = df[df['banner_id0'] == df['banner_id']]
df.head()

CPU times: user 51.9 s, sys: 15.8 s, total: 1min 7s
Wall time: 1min 20s


,date_time,zone_id,banner_id,campaign_clicks,os_id,country_id,banner_id0,g0,coeff_sum0,banner_id1,g1,coeff_sum1,impressions,clicks
1,2021-09-26 22:54:49.000000,1,1,0,0,1,1,0.054298,-2.657477,269,0.031942,-4.449220,1,1
2,2021-09-26 23:57:20.000000,2,2,3,0,0,2,0.014096,-3.824875,21,0.014906,-3.939309,1,1
3,2021-09-27 00:04:30.000000,3,3,0,1,1,3,0.015232,-3.461357,99,0.050671,-3.418403,1,1
4,2021-09-27 00:06:21.000000,4,4,0,1,0,4,0.051265,-4.009026,11464230,0.032005,-2.828797,1,1
5,2021-09-27 00:06:50.000000,5,5,0,2,2,5,0.337634,-3.222757,37,0.338195,-3.221755,1,1


In [5]:
def analysis(df: pd.DataFrame):
    # посчитаем количество NaN в стобцах
    print(f'Чисто NaN в данных: {df.isna().sum().sum()}', '\n')

    print('Посчитаем количество уникальных признаков в каждом столбце:')
    for name in df.columns.values[1:-1]:
      print(name.ljust(20), df[name].nunique())
    print()

    print('Посмотрим на распределение дней:')
    days = df.date_time.apply(lambda x: x[:10])
    days_counts = days.value_counts()
    print(days_counts.sort_index(), '\n')

In [6]:
%%time
analysis(df)

Чисто NaN в данных: 39350 

Посчитаем количество уникальных признаков в каждом столбце:
zone_id              3302
banner_id            1606
campaign_clicks      822
os_id                11
country_id           17
banner_id0           1606
g0                   13637451
coeff_sum0           4809216
banner_id1           3140906
g1                   13394604
coeff_sum1           5244097
impressions          1

Посмотрим на распределение дней:
2021-09-01          1
2021-09-26    2707350
2021-09-27    2083239
2021-09-28    2028991
2021-09-29    2153388
2021-09-30    1640889
2021-10-01    1442740
2021-10-02    1890562
Name: date_time, dtype: int64 

CPU times: user 22 s, sys: 1.21 s, total: 23.2 s
Wall time: 23.2 s


Выводы по данным:
* В данных есть NaN-ы, они в параметрах g0, g1 и тп.
* В данных есть колонка `impressions`, у которой одно значение, она не вносит никакого вклада, ее уберем
* Среди `date_time` есть день с одной записью, по самим данным он не выглядит как что-то из ряда вон выходящее, поэтому его можно оставить, хотя его существование овеяно тайной и загадкой :)
* Дни по сути представлены одной неделей, в ответе нам надо предоставить результаты для `2021-10-02`.
* Я думаю, что вводить признак дня недели тоже имеет смысл, но я не буду этого делать, потому что в test день недели, который не попадает в train, и этот признак не сможем проверить. При прочих равных этот признак я бы добавил.
* Добавим в качестве признака час просмотра рекламы. Эта фича может быть полезной - в зависимости от времени посещаются те или иные сайты, соответственно, появляется разная реклама. 
* Признак `campaign_clicks` больше не нужен, потому что у нас есть информация для banner_id (теперь тоже самое, что и banner_id1), а для banner_id1 - нет, поэтому дропнем и `campaign_clicks` 
* Остальные признаки закодируем с помощью OneHotEncoding




In [7]:
def feature_engineering(df: pd.DataFrame) -> pd.DataFrame:

    # уберем строки с пропусками
    df = df[~df['g1'].isna()]
    
    # таблица с параметрами
    df_param = df[['banner_id0', 'banner_id1', 'g0', 'g1', 'coeff_sum0', 'coeff_sum1']].copy()
    df.drop(['banner_id0', 'banner_id1', 'g0', 'g1', 'coeff_sum0', 'coeff_sum1'], axis=1, inplace=True)

    # добавим признак времени
    df['date_time'] = pd.to_datetime(df['date_time'])
    df['hour'] = df['date_time'].dt.hour
    
    # вытащим нужные признаки и дропнем их таблицы
    clicks = df['clicks'].to_numpy()
    idx_train = df['date_time'] < pd.to_datetime('2021-10-02')

    # campaign_clicks здесь больше не нужна
    df.drop(['date_time', 'impressions', 'clicks', 'campaign_clicks'], axis=1, inplace=True)

    # разделим на train и test
    X_train, y_train = df[idx_train].to_numpy(), clicks[idx_train]
    X_test, y_test = df[~idx_train].to_numpy(), clicks[~idx_train]

    # колдуем и кодируем
    enc = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=True)    
    X_train = enc.fit_transform(X_train)
    X_test = enc.transform(X_test)

    # данные, чтобы посчитать coeff_sum1_new - меняем banner_id на banner_id1
    df['banner_id'] = df_param['banner_id1']
    X_test_1 = df[~idx_train].to_numpy()
    X_test_1 = enc.transform(X_test_1)

    return X_train, y_train, X_test, X_test_1, y_test, df_param[~idx_train]

In [8]:
%%time
X_train, y_train, X_test, X_test_1, y_test, df_param = feature_engineering(df)

print(f'Число записей в train: {X_train.shape[0]}')
print(f'Число записей в test : {X_test.shape[0]}')
print(f'Число признаков: {X_train.shape[1]}')

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-7-894cde8e8509>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date_time'] = pd.to_datetime(df['date_time'])
<ipython-input-7-894cde8e8509>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

Число записей в train: 12041815
Число записей в test : 1885670
Число признаков: 4785
CPU times: user 30.7 s, sys: 1.35 s, total: 32.1 s
Wall time: 33.9 s


In [9]:
df_param

,banner_id0,banner_id1,g0,g1,coeff_sum0,coeff_sum1
164,76,401,0.055551,0.030272,-2.926980,-3.390642
166,46,11464251,0.017521,0.085038,-1.377320,-3.329596
168,76,11464252,0.171074,0.079034,-3.112081,-1.907685
169,46,0,0.017439,0.017624,-2.493974,-3.889516
359,2,49,0.020414,0.068041,-2.154111,-3.088063
...,...,...,...,...,...,...
15821452,89,7,0.043305,0.014551,-2.983741,-3.074290
15821455,132,14623598,0.031065,0.099036,-3.745272,-2.287568
15821461,52,14623601,0.008300,0.028516,-3.538011,-2.011854
15821467,530,481,0.040037,0.039634,-3.495224,-3.493091


In [10]:
def create_model(penalty='l2', solver='liblinear', C=1):
    model = LogisticRegression(penalty=penalty, solver=solver, C=C)
    return model

In [11]:
%%time
np.random.seed(42)
model = create_model(C=0.01)
model.fit(X_train, y_train)

y_pred = model.predict_proba(X_test)[:, 1]

loss = log_loss(y_test, y_pred)
roc_score = roc_auc_score(y_test, y_pred)
print(f"Log loss: {loss}, roc-auc: {roc_score}")

Log loss: 0.13431090491111758, roc-auc: 0.7875953324218785
CPU times: user 1min 12s, sys: 1.02 s, total: 1min 13s
Wall time: 1min 14s


Выбор баннера определяется с помощью вероятностей $\pi_0$ и $\pi_1$, которые получаются одинаковым образом через сравнение двух нормальных случайных величин (на примере $\pi_0$):

$$x\sim N(\text{coeff_sum0}, g0^2),\ y\sim N(\text{coeff_sum1}, g1^2)$$
тогда
$$\xi = y-x\sim N(\mu, \sigma^2),$$
где
$$\mu = \text{coeff_sum1} - \text{coeff_sum0}$$
$$\sigma^2 = g0^2 + g1^2.$$

Следовательно,
$$ \pi_0 = P(x>y) = P(y-x<0) = P(\xi<0) = \Phi\left(\frac{0-\mu}{\sigma}\right)$$

Аналогично для $\pi_1$


In [12]:
from scipy.special import logit
from scipy.stats import norm

In [13]:
def count_pi(mu0, mu1, sigma0, sigma1):
    mu = -mu1 + mu0
    sigma = np.sqrt(sigma0 ** 2 + sigma1 ** 2)
    return norm.cdf(mu / sigma)

In [14]:
df_param['coeff_sum0_new'] = logit(model.predict_proba(X_test)[:, 1])
df_param['coeff_sum1_new'] = logit(model.predict_proba(X_test_1)[:, 1])

In [15]:
df_param['pi_0'] = count_pi(df_param['coeff_sum0'],
                            df_param['coeff_sum1'],
                            df_param['g0'],
                            df_param['g1'])

df_param['pi_1'] = count_pi(df_param['coeff_sum0_new'],
                            df_param['coeff_sum1_new'],
                            df_param['g0'],
                            df_param['g1'])

In [16]:
lambda_ = 10
cips = np.mean(y_test * np.clip(df_param['pi_1'] / df_param['pi_0'], a_min=None, a_max=lambda_))
print(f'CIPS_score: {cips}')

CIPS_score: 0.07759982286077578
